In [40]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from tqdm import tqdm
from mpl_toolkits import mplot3d
from numpy import array, where, shape, reshape

In [41]:
file = uproot.open("/sno/py_out1.root")

In [42]:
def EV_info(file ,evID):
    #function which extracts info. of an event given the event ID
    
    import uproot
    import numpy as np
    from numpy import array, where, shape, reshape
    
    #Load file
    #file = uproot.open("/sno/py_out1.root")
    #EV info
    data1 = file['T;7']
    
    evtid = np.array(data1['evtid'])
    mc_position = np.array(data1['mc_position'])
    position = np.array(data1['position'])
    hit_pmtid = np.array(data1['hit_pmtid'])
    hit_residual = np.array(data1['hit_residual'])
    hit_type = np.array(data1['hit_type'])
    
    #Extract the event given the evtid----------------------
    condition = (evID == evtid)
    
    #if condition satisfied, extract the information in the index where the condition is True
    mc_position_ev = np.extract(condition, mc_position)
    position_ev = np.extract(condition, position)
    residual_ev = np.extract(condition, hit_residual)
    type_e = np.extract(condition, hit_type)
    
    #Indices for which elements are True in eventID then extract the hit_pmtID and save in pmtif_ev
    pmtid_ev = np.extract(condition, hit_pmtid) 
    
    return pmtid_ev

In [43]:
def clean_pmt_type(file, type_, evID):
    
    #clean_pmt <=> list of PMT id and index of the pmt_type desired
    #Function that receive the type of PMT desired to clean the PMT type for a given event ID (evID).
    
    # Receive the pmtid of the hit pmt given an event and return the id and index (onto PMT info) of the PMT 
    # such that the coordinate of PMT can be extracted.
    
    import uproot
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sn
    from tqdm import tqdm
    from mpl_toolkits import mplot3d
    from numpy import array, where, shape, reshape
    
    #upload file
    #file = uproot.open("/sno/py_out1.root")
    
    #pmt_info
    pmt_info = file['pmt;1']
    
    pmt_id = array(pmt_info['pmt_id'])
    #pmt_pos_xyz = array(pmt_info['pmt_pos_xyz'])
    #pmt_pos_sph = array(pmt_info['pmt_pos_sph'])
    pmt_type = array(pmt_info['pmt_type'])
    
    #selection of PMT type-------------------------------------
    clean_pmt_id = np.array([],np.int32)                   #pmtID of the desired type
    valid_index_pmt_type = np.array([],np.int32)           #index of the desired PMTs for the type required
    
    for type_i in type_:
        p = where(pmt_type == type_i)[0]  # p get the indices where the PMT_type are for each type_ desired
        valid_index_pmt_type = np.concatenate((valid_index_pmt_type,p))
        
    for valid_i in valid_index_pmt_type:
        clean_pmt_id = np.append(clean_pmt_id,pmt_id[valid_i])
        

    # Extract the indices and ID of the clean PMT given the event-------------------
    
    pmtid_ev = EV_info(file, evID)                                    #Get the array of pmtID which verify a hit
    
    pmt_clean_id_i = where(np.in1d(clean_pmt_id, pmtid_ev))[0]        # Get the index on the PMT info when the PMT cleaned list is in the PMT ID event
    
    return pmt_clean_id_i, clean_pmt_id

In [44]:
 , clean_pmt_id = clean_pmt_type(file, [1,7], 14)
#print(PMT_i_clean)
print(clean_pmt_id)

[   1    2    3 ... 3536 3856 8163]


In [48]:
def GetCoordinateshitPMT(file, type_, evID):
    
    #Function that receives the type of PMT and event ID of interest and return the PMT coordinates
    #ruturn PMT coordinates in [(x,y,z), (tetha,phi,r)]
    
    #Load file
    file = uproot.open("/sno/py_out1.root")
    pmt_info = file['pmt;1']
    
    pmt_id = array(pmt_info['pmt_id'])
    pmt_pos_xyz = array(pmt_info['pmt_pos_xyz'])
    pmt_pos_sph = array(pmt_info['pmt_pos_sph'])
    
    hitpmt_coordxyz = []
    hitpmt_coordsphe = []
    
    #Look fot the clean PMT IDs from clean_pmt_type function
    
    pmt_clean_id_i ,clean_pmt_id = clean_pmt_type(file, type_, evID)
    
    for i in pmt_clean_id_i:
        hitpmt_coordxyz.append(pmt_pos_xyz[clean_pmt_id[i]])
        hitpmt_coordsphe.append(pmt_pos_sph[clean_pmt_id[i]])
        
    #arange of information
    
    
        
    return array(hitpmt_coordxyz), array(hitpmt_coordsphe)
    

In [49]:
hitpmt_coordxyz,_ = GetCoordinateshitPMT(file, [1,7], 15)

In [50]:
hitpmt_coordxyz

array([[ 5467.54,    63.44, -6406.27],
       [ 5615.16,   266.8 , -6251.09],
       [ 5762.73,   469.91, -6095.92],
       ...,
       [-4287.58,  6152.61,  3796.11],
       [-6600.28,  3114.85,  4197.48],
       [ 2545.4 , -7078.81,  3737.76]])

In [30]:
%matplotlib

fig = plt.figure(figsize = (10,10))
ax = plt.axes(projection='3d')
ax.grid()

x = hitpmt_coordxyz[:,0]
y = hitpmt_coordxyz[:,1]
z = hitpmt_coordxyz[:,2]

ax.scatter(x, y, z, c = 'r', s = 1)

Using matplotlib backend: Qt5Agg


## Plot functions

In [ ]:
def hist(x):
    
    import seaborn as sn
    
    
    #histogram function
    
    plt.figure(figsize=(8,6))
    sn.histplot(x, bins = 100)
    plt.xlabel('hit_time_residual')
    plt.ylabel('events')
    plt.yscale('log')